In [1]:
from collections import Counter
from functools import partial

import pandas as pd
import numpy as np
import scipy

from statsmodels.formula.api import ols

from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML, clear_output

%matplotlib inline

In [2]:
from sklearn.cross_validation import StratifiedShuffleSplit, ShuffleSplit
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.decomposition import TruncatedSVD

In [3]:
from libscores import bac_metric

In [4]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam
from keras.regularizers import WeightRegularizer, l1, l2
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU, ELU, LeakyReLU
from keras.callbacks import EarlyStopping, Callback

Using Theano backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [5]:
class WatchlistCallback(Callback):
    def __init__(self, watchlist, eval_metric, epoch_no=10):
        super(Callback, self).__init__()

        self.epoch_no = epoch_no
        self.X, self.y = watchlist
        self.eval_metric = eval_metric
        self.scores = []
        self.epochs = []

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.epoch_no == 0:
            y_pred = self.model.predict(self.X, verbose=0)
            score = self.eval_metric(self.y, y_pred)
            self.scores.append(score)
            self.epochs.append(epoch)

            print "score: %0.6f (epoch no %d)" % (score, epoch)  

class StatusCallback(Callback):    
    def __init__(self, epoch_total):
        super(Callback, self).__init__()
        self.progress = tqdm(total=epoch_total)
        
    def on_epoch_end(self, epoch, logs={}):
        self.progress.update(1)

    def __enter__(self): 
        return self

    def __exit__(self, type, value, traceback):
        self.progress.close()

In [6]:
def read_info(file_name):
    result = []
    
    for line in file(file_name):
        key, value = line.strip().split('=')
        key = key.strip()
        value = value.strip().strip("'")
        if value.isdigit():
            value = int(value)
        result.append((key, value))
    
    return dict(result)

In [7]:
public = read_info('data/set3_helena/helena_public.info')
public

{'feat_num': 27,
 'feat_type': 'Numerical',
 'has_categorical': 0,
 'has_missing': 0,
 'is_sparse': 0,
 'label_num': 100,
 'metric': 'bac_metric',
 'name': 'helena',
 'target_num': 100,
 'target_type': 'Categorical',
 'task': 'multiclass.classification',
 'test_num': 18628,
 'time_budget': 1200,
 'train_num': 65196,
 'usage': 'AutoML challenge 2014',
 'valid_num': 9314}

In [8]:
def read_float_dense_features(file_name):
    result = list()
    for line in file(file_name):
        row = [float(f) for f in line.strip().split(' ')]
        result.append(row)
    return np.array(result)

In [9]:
def read_multilabels(file_name):
    result = list()
    for line in file(file_name):
        row = [f for f in line.strip().split(' ')]
        result.append(row.index('1'))
    return np.array(result)

In [10]:
X = read_float_dense_features('data/set3_helena/helena_train.data')
y = read_multilabels('data/set3_helena/helena_train.solution')

X_valid = read_float_dense_features('data/set3_helena/helena_valid.data')
X_test  = read_float_dense_features('data/set3_helena/helena_test.data')

print X.shape, y.shape, np.unique(y).shape
print X_valid.shape, X_test.shape

(65196, 27) (65196,) (100,)
(9314, 27) (18628, 27)


In [11]:
classes = np.unique(y)
y_bin = label_binarize(y, classes=classes)

In [12]:
scaler = StandardScaler(with_mean=True, copy=False)
X = scaler.fit_transform(X)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [13]:
train_size = 0.8
EPOCHS = 500

cv = StratifiedShuffleSplit(y, n_iter=1, train_size=train_size, random_state=1)
train, test = next(cv.__iter__())

In [53]:
model = Sequential()

model.add(Dense(input_dim=X.shape[1], output_dim=300, init='uniform', W_regularizer=l2(0.005))) 
model.add(Activation(PReLU()))
model.add(BatchNormalization())
# model.add(Dropout(0.01))

model.add(Dense(input_dim=300, output_dim=100, init='uniform', W_regularizer=l1(0.01))) 
model.add(Activation(PReLU())) 
model.add(BatchNormalization())
#model.add(Dropout(0.1)) 

model.add(Dense(input_dim=100, output_dim=y_bin.shape[1], init='uniform', W_regularizer=l2(0.001))) 
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

eval_metric = partial(bac_metric, task='multiclass.classification')
watchlist = WatchlistCallback(watchlist=(X[test], y_bin[test]), eval_metric=eval_metric, epoch_no=1)

In [54]:
model.fit(X[train], y_bin[train], nb_epoch=EPOCHS, batch_size=10000, 
          callbacks=[watchlist], verbose=0)

score: 0.032388 (epoch no 0)
score: 0.056329 (epoch no 1)
score: 0.065575 (epoch no 2)
score: 0.072816 (epoch no 3)
score: 0.076153 (epoch no 4)
score: 0.081390 (epoch no 5)
score: 0.087147 (epoch no 6)
score: 0.088382 (epoch no 7)
score: 0.089120 (epoch no 8)
score: 0.099444 (epoch no 9)
score: 0.107322 (epoch no 10)
score: 0.104703 (epoch no 11)
score: 0.113807 (epoch no 12)
score: 0.117815 (epoch no 13)
score: 0.118429 (epoch no 14)
score: 0.120909 (epoch no 15)
score: 0.131678 (epoch no 16)
score: 0.132953 (epoch no 17)
score: 0.126012 (epoch no 18)
score: 0.136867 (epoch no 19)
score: 0.142015 (epoch no 20)
score: 0.137260 (epoch no 21)
score: 0.145232 (epoch no 22)
score: 0.141026 (epoch no 23)
score: 0.145159 (epoch no 24)
score: 0.139783 (epoch no 25)
score: 0.155665 (epoch no 26)
score: 0.155782 (epoch no 27)
score: 0.161756 (epoch no 28)
score: 0.154389 (epoch no 29)
score: 0.160963 (epoch no 30)
score: 0.165869 (epoch no 31)
score: 0.170698 (epoch no 32)
score: 0.164494 (epo

In [54]:
model = Sequential()

model.add(Dense(input_dim=X_input.shape[1], output_dim=300, init='uniform')) 
model.add(Activation('linear')) 
model.add(Dropout(0.01)) 

model.add(Dense(input_dim=300, output_dim=100, init='uniform')) 
model.add(Activation('tanh')) 
model.add(Dropout(0.1)) 

model.add(Dense(input_dim=100, output_dim=y_bin.shape[1], init='uniform')) 
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

EPOCHS=224

with StatusCallback(epoch_total=EPOCHS) as status:
    model.fit(X_input, y_bin, nb_epoch=EPOCHS, batch_size=10000, callbacks=[status], verbose=0)

In [55]:
y_valid_score = model.predict(X_valid)
y_test_score = model.predict(X_test)

In [56]:
np.savetxt('submission/helena_valid.predict', y_valid_score, fmt='%0.10f')
np.savetxt('submission/helena_test.predict', y_test_score, fmt='%0.10f')